In [2]:
import numpy as np
import pandas as pd
import os


### WORKFLOW

1) Reading csv files 
2) Parse fields including sptail grid and maneuver labels
3) Using unique vehicle ids to spilit train(70%)/validation(10%)/test(20%)
4) Only reserve those data sample with at least 3s history and 5s future

#### Hyperparameters:

- 30ms for history traj
- 50ms for future traj
- past and future 40ms for lateral behavior detaction.
- grid is splitted with size 25x5 (2.5x2 meters for each grid) OR 25x7 (2.5x1.5 meters for each grid)

#### Fields in the final result:
##### traj  : (data number)*(13+grid_num)
1) Dataset Id
2) Vehicle Id
(Column in tracks)
3) Frame Id
4) Local X
5) Local Y
6)  Lane Id
7*) Lateral maneuver
8*) Longitudinal maneuver
9 ) Length
10) Width
11) Class label
12) Velocity
13) Accerlation
14-end*) Neighbor Car Ids at grid location


##### tracks: includes {Dataset_Id*Vehicle_Id}, each cell with (11+grid)*totalFramNum

1) Frame Id
2) Local X
3)  Local Y
4)  Lane Id
5)  Lateral maneuver
6) Longitudinal maneuver
7) Length
8) Width
9) Class label
10) Velocity
11) Accerlation
12-end*) Neighbor Car Ids at grid location

In [7]:
# Set hyperparameters
grid_length=25; grid_width=5; cell_length=2; cell_width=1.5;

# Save location
raw_folder = f'../dataset/pNEUMA/{grid_length}x{grid_width}_raw/'
post_folder = f'../dataset/pNEUMA/{grid_length}x{grid_width}/'
os.makedirs(raw_folder, exist_ok=True)
os.makedirs(post_folder, exist_ok=True)

# Other variable dependent on grid.
grid_cells = grid_length * grid_width
grid_cent_location = int(grid_length*grid_width*0.5 + 0.5)



### 1. Load data


In [4]:
def preprocess_pneuma(csv_k, meta_k):
    # Remove the the row that does not contain "Lane" in "Traffic Regions(list)" column
    csv_k = csv_k[csv_k['Traffic Regions(list)'].str.contains('Lane')]
    # Remove the row "Off Lane" in "Traffic Regions(list)" column
    csv_k = csv_k[~csv_k['Traffic Regions(list)'].str.contains('Off Lane')]
    # Remove "Lane" from "Traffic Regions(list)" column
    csv_k['Traffic Regions(list)'] = csv_k['Traffic Regions(list)'].str.replace('Lane', '').astype('int')
    # Drop the column "nearest_point_on_lane_center"
    csv_k = csv_k.drop(columns=['nearest_point_on_lane_center'])
    
    # Append vehicle class column to csv_k from meta_k
    csv_k['v_Class'] = csv_k['Track ID'].map(meta_k.set_index('Track ID')['Type'])
    # Class to int
    csv_k['v_Class'] = csv_k['v_Class'].map({' Car': 1, ' Taxi':2, 
                                            ' Medium Vehicle':3, ' Heavy Vehicle':3,  
                                            ' Bus':4, 
                                            ' Motorcycle':5, ' Bicycle':-1})

    # Vehicle width and length by class (in meter)
    csv_k['v_Width'] = csv_k['v_Class'].map({1:2.5, 2:2.5, 3:3.5, 4:3.5, 5:1.5, -1:1.5})
    csv_k['v_Length'] = csv_k['v_Class'].map({1:5.0, 2:5.0, 3:7.0, 4:12.0, 5:2.0, -1:2.0})
    # Drop the row that is Bicycle
    csv_k = csv_k[csv_k['v_Class'] != -1]

    # Filter the row every 10th frame (5fps)
    csv_k = csv_k.loc[(csv_k['Time [s]']*10e6).astype(int) % 10e5 == 0].reset_index(drop=True)
    # Convert time to frame index
    csv_k['Frame index'] = csv_k['Time [s]']/0.2

    # convert speed to m/s
    csv_k['Velocity'] = csv_k['Speed [km/h]'] * 1000 / 3600
    # compute acceleration
    csv_k['Acceleration'] = csv_k.groupby('Track ID')['Velocity'].diff() /0.2
    # Drop the row where Acceleration is NaN
    csv_k = csv_k.dropna(subset=['Acceleration']).reset_index(drop=True)
    
    # Rename the column
    csv_k = csv_k.rename(columns={'Track ID': 'Vehicle id',
                                    'lateral_dist': 'Local X', 'longitudinal_dist': 'Local Y',
                                    'Traffic Regions(list)': 'Lane id', 
                                    'Angle [rad]': 'azimuth',
                                })       

    # Long Lat Manuever
    csv_k = append_lat_long_manuever(csv_k)

    # Order the column acoording to the final dataset
    # (1:dataset id,) 2:Vehicle id, 3:Frame index,
    # 4:Local X, 5:Local Y, 6:Lane id.
    # 7: Lateral manuever, 8: Longitudinal manuever, 
    # 9:v_length, 10:v_Width, 11:v_Class
    # 12:Velocity (m/s), 13:Acceleration (m/s2).
    # 14: azimuth (rad)
    csv_k = csv_k[['Vehicle id', 'Frame index', 
                    'Local X', 'Local Y', 'Lane id',
                    'Lateral_manuever', 'Longitudinal_manuever',
                    'v_Length', 'v_Width', 'v_Class',
                    'Velocity', 'Acceleration', 'azimuth']]
    

    return csv_k

def append_lat_long_manuever(csv_k):
    # For every vehicle id, 
    # 1. find when they change lane and assign the lane change label (2: left, 3: right)
    # 2. find their longitudinal manuever and assign the label (1: normal, 2: break)

    res_df = pd.DataFrame()
    csv_k['Lateral_manuever'] = 1
    for vid in csv_k['Vehicle id'].unique():
        csv_k_vid = csv_k[csv_k['Vehicle id'] == vid].reset_index(drop=True)
        # 1. Find the lane change
        lane_change = csv_k_vid['Lane id'].diff().fillna(0)
        # Find the lane change index
        lane_change_idx = lane_change[lane_change != 0].index
        # Find the lane change direction
        lane_change_dir = lane_change[lane_change != 0].values
        # Assign the lane change label (+- 15 frames = 3 seconds)
        for i in range(len(lane_change_idx)):
            # Left lane change
            if lane_change_dir[i] == -1:
                csv_k_vid.loc[lane_change_idx[i]-15:lane_change_idx[i]+15, 'Lateral_manuever'] = 2
            # Right lane change
            elif lane_change_dir[i] == 1:
                csv_k_vid.loc[lane_change_idx[i]-15:lane_change_idx[i]+15, 'Lateral_manuever'] = 3

        # 2. Find the longitudinal manuever
        # Take the historical time window (10 frames = 2 seconds) and the future time window (20 frames = 4 seconds)
        hist_k = csv_k_vid[['Local Y', 'Frame index']].shift(10).fillna(method='bfill')
        fut_k = csv_k_vid[['Local Y', 'Frame index']].shift(-20).fillna(method='ffill')
        # compute the average velocity
        csv_k_vid['vHist'] = (hist_k['Local Y'] - csv_k_vid['Local Y'])/(hist_k['Frame index'] - csv_k_vid['Frame index'])/0.2
        csv_k_vid['vFut'] = (csv_k_vid['Local Y'] - fut_k['Local Y'])/(csv_k_vid['Frame index'] - fut_k['Frame index'])/0.2
        # assign the longitudinal manuever label: vFut/vHist <0.8 (1 : normal, 2: break )
        csv_k_vid['Longitudinal_manuever'] = 1
        csv_k_vid.loc[np.abs(csv_k_vid['vFut']/csv_k_vid['vHist'])  < 0.8, 'Longitudinal_manuever'] = 2

        # Append the result
        res_df = pd.concat([res_df, csv_k_vid], axis=0)

    return res_df


In [6]:
dataset_name = f'../../DATASETS/PNEUMA/pneuma_raw/monday_d8_1000_1030_traj.csv'
meta_name = f'../../DATASETS/PNEUMA/pneuma_raw/monday_d8_1000_1030_meta.csv'
csv_k = pd.read_csv(dataset_name)
meta_k = pd.read_csv(meta_name)
csv_k = preprocess_pneuma(csv_k, meta_k)
csv_k.head()

,Vehicle id,Frame index,Local X,Local Y,Lane id,Lateral_manuever,Longitudinal_manuever,v_Length,v_Width,v_Class,Velocity,Acceleration,azimuth
0,46,37.0,-5.777249,337.123047,3,1,1,5.0,2.5,1,7.030944,-1.476111,2.8169
1,46,38.0,-6.022306,338.451372,3,1,1,5.0,2.5,1,6.798306,-1.163194,2.7756
2,46,39.0,-6.315036,339.736949,3,1,1,5.0,2.5,1,6.619528,-0.893889,2.7258
3,46,40.0,-6.655067,341.214288,3,1,1,5.0,2.5,1,6.490972,-0.642778,2.6694
0,47,136.0,-2.793493,338.237904,2,1,1,7.0,3.5,3,10.730361,-0.639583,2.9452


#### test

In [9]:
otherVehsInRange


,Vehicle id,Local X,Local Y,v_Class,grid_locX,grid_locY,grid_loc
555,48,-0.400151,445.926176,1,5.0,2.0,102.0
538,171,-3.584659,453.152658,2,3.0,6.0,56.0
613,221,-5.854546,455.161050,1,1.0,7.0,7.0
586,231,-5.869481,463.865792,2,1.0,11.0,11.0
561,242,-3.609280,446.301213,1,3.0,3.0,53.0
502,262,-1.986256,456.509621,5,4.0,8.0,83.0
242,344,-1.811093,449.145160,5,4.0,4.0,79.0


In [205]:
print(grid_loc)
otherVehsInRange.loc[otherVehsInRange['grid_loc'] == grid_loc]['Vehicle id'].values[0]

56.0


207

In [19]:
csv_k_vid.columns

Index(['Vehicle id', 'Frame index', 'Local X', 'Local Y', 'Lane id',
       'Lateral_manuever', 'Longitudinal_manuever', 'v_Length', 'v_Width',
       'v_Class',
       ...
       'grid_115', 'grid_116', 'grid_117', 'grid_118', 'grid_119', 'grid_120',
       'grid_121', 'grid_122', 'grid_123', 'grid_124'],
      dtype='object', length=142)

In [ ]:
def get_other_vehicles_in_grid_range(vid, csv_k):
    csv_k_vid = csv_k[csv_k['Vehicle id'] == vid].reset_index(drop=True)

    # Get grid locations in Column 14~13+grid_length*grid_width:
    csv_k_vid['centVehX'] = csv_k_vid['Local X']
    csv_k_vid['centVehY'] = csv_k_vid['Local Y']
    csv_k_vid['gridMinX'] = csv_k_vid['centVehX'] - 0.5 * grid_width * cell_width
    csv_k_vid['gridMinY'] = csv_k_vid['centVehY'] - 0.5 * grid_length * cell_length

    #Create a columns that will store the vehicle id of the other vehicles in the grid
    grid_array = np.zeros((csv_k_vid.shape[0], grid_length*grid_width))
    grid_df = pd.DataFrame(grid_array, columns=[f'grid_{i}' for i in range(grid_length*grid_width)])
    csv_k_vid = pd.concat([csv_k_vid, grid_df], axis=1)

    # Find other vehicles at the same time from csv_k
    for time in csv_k_vid['Frame index']:
        grid_loc_list = np.zeros(grid_length*grid_width)
        centVehX = csv_k_vid[csv_k_vid['Frame index'] == time]['centVehX'].values[0]
        centVehY = csv_k_vid[csv_k_vid['Frame index'] == time]['centVehY'].values[0]
        gridMinX = csv_k_vid[csv_k_vid['Frame index'] == time]['gridMinX'].values[0]
        gridMinY = csv_k_vid[csv_k_vid['Frame index'] == time]['gridMinY'].values[0]
        otherVehsAtTime = csv_k[csv_k['Frame index'] == time][['Vehicle id', 'Local X', 'Local Y', 'v_Class']]
        otherVehsInRange = otherVehsAtTime[(np.abs(otherVehsAtTime['Local Y'] - centVehY) < 0.5 * grid_length * cell_length) &
                                            (np.abs(otherVehsAtTime['Local X'] - centVehX) < 0.5 * grid_width * cell_width)].copy()
        
        # print(centVehX, centVehY, gridMinX, gridMinY)
        if otherVehsInRange.size > 0:
            otherVehsInRange['grid_locX'] = otherVehsInRange.apply(lambda x: np.ceil((x['Local X'] - gridMinX) / float(cell_width)), axis=1)
            otherVehsInRange['grid_locY'] = otherVehsInRange.apply(lambda x: np.ceil((x['Local Y'] - gridMinY) / float(cell_length)), axis=1)
            otherVehsInRange['grid_loc'] = otherVehsInRange.apply(lambda x: x['grid_locY'] + (x['grid_locX'] - 1) * grid_length, axis=1)

            # Drop the vehicles whose grid location is grid_cent_location
            otherVehsInRange = otherVehsInRange[otherVehsInRange['grid_loc'] != grid_cent_location]

            # make the list of where grid_loc is and what vehicle id is
            for grid_loc in otherVehsInRange['grid_loc'].unique():
                grid_loc_list[int(grid_loc)-1] = otherVehsInRange.loc[otherVehsInRange['grid_loc'] == grid_loc]['Vehicle id'].values[0]
            
            # Overwrite the grid_loc_list to the rows of csv_k_vid dataframe from the grid_0 columns
            csv_k_vid.loc[csv_k_vid['Frame index'] == time, f'grid_0':f'grid_{grid_length*grid_width-1}'] = grid_loc_list

            print(time, grid_loc_list)

    # Remove auxiliary columns
    csv_k_vid = csv_k_vid.drop(['centVehX', 'centVehY', 'gridMinX', 'gridMinY'], axis=1)
    
    return csv_k_vid

In [20]:
csv_k_vid = csv_k[csv_k['Vehicle id'] == 47].reset_index(drop=True)

# Get grid locations in Column 14~13+grid_length*grid_width:
csv_k_vid['centVehX'] = csv_k_vid['Local X']
csv_k_vid['centVehY'] = csv_k_vid['Local Y']
csv_k_vid['gridMinX'] = csv_k_vid['centVehX'] - 0.5 * grid_width * cell_width
csv_k_vid['gridMinY'] = csv_k_vid['centVehY'] - 0.5 * grid_length * cell_length

#Create a columns that will store the vehicle id of the other vehicles in the grid
grid_array = np.zeros((csv_k_vid.shape[0], grid_length*grid_width))
grid_df = pd.DataFrame(grid_array, columns=[f'grid_{i}' for i in range(grid_length*grid_width)])
csv_k_vid = pd.concat([csv_k_vid, grid_df], axis=1)

# Find other vehicles at the same time from csv_k
for time in csv_k_vid['Frame index']:
    grid_loc_list = np.zeros(grid_length*grid_width)
    centVehX = csv_k_vid[csv_k_vid['Frame index'] == time]['centVehX'].values[0]
    centVehY = csv_k_vid[csv_k_vid['Frame index'] == time]['centVehY'].values[0]
    gridMinX = csv_k_vid[csv_k_vid['Frame index'] == time]['gridMinX'].values[0]
    gridMinY = csv_k_vid[csv_k_vid['Frame index'] == time]['gridMinY'].values[0]
    otherVehsAtTime = csv_k[csv_k['Frame index'] == time][['Vehicle id', 'Local X', 'Local Y', 'v_Class']]
    otherVehsInRange = otherVehsAtTime[(np.abs(otherVehsAtTime['Local Y'] - centVehY) < 0.5 * grid_length * cell_length) &
                                        (np.abs(otherVehsAtTime['Local X'] - centVehX) < 0.5 * grid_width * cell_width)].copy()
    
    # print(centVehX, centVehY, gridMinX, gridMinY)
    if otherVehsInRange.size > 0:
        otherVehsInRange['grid_locX'] = otherVehsInRange.apply(lambda x: np.ceil((x['Local X'] - gridMinX) / float(cell_width)), axis=1)
        otherVehsInRange['grid_locY'] = otherVehsInRange.apply(lambda x: np.ceil((x['Local Y'] - gridMinY) / float(cell_length)), axis=1)
        otherVehsInRange['grid_loc'] = otherVehsInRange.apply(lambda x: x['grid_locY'] + (x['grid_locX'] - 1) * grid_length, axis=1)

        # Drop the vehicles whose grid location is grid_cent_location
        otherVehsInRange = otherVehsInRange[otherVehsInRange['grid_loc'] != grid_cent_location]

        # make the list of where grid_loc is and what vehicle id is
        for grid_loc in otherVehsInRange['grid_loc'].unique():
            grid_loc_list[int(grid_loc)-1] = otherVehsInRange.loc[otherVehsInRange['grid_loc'] == grid_loc]['Vehicle id'].values[0]
        
        # Overwrite the grid_loc_list to the rows of csv_k_vid dataframe from the grid_0 columns
        csv_k_vid.loc[csv_k_vid['Frame index'] == time, f'grid_0':f'grid_{grid_length*grid_width-1}'] = grid_loc_list

        print(time, grid_loc_list)
        # # Put the vehicle id according to the grid location
        # for grid_loc in otherVehsInRange['grid_loc'].unique():
        #     csv_k_vid.loc[csv_k_vid['Frame index'] == time, f'grid_{grid_loc}'] = otherVehsInRange[otherVehsInRange['grid_loc'] == grid_loc]['Vehicle id'].values
            
        # # make dictionary of {grid_loc: Vehicle id}
        # neighbors = otherVehsInRange[['grid_loc', 'Vehicle id']].set_index('grid_loc').T.to_dict('list')
        # print(time, neighbors, csv_k_vid.loc[csv_k_vid['Frame index'] == time])
        # # Append the neighbor information to csv_k_vid
        # csv_k_vid.loc[csv_k_vid['Frame index'] == time, 'neighbors'] = neighbors
    # print(otherVehsInRange, neighbors)

    #     # Find the grid location for each vehicle
    #     for neighbor_vid in otherVehsInRange['Vehicle id'].unique():
    #         otherVehsInRange[otherVehsInRange['Vehicle id'] == neighbor_vid]['gridLoc'] = otherVehsInRange['Local Y'].values[0]
    #         print(otherVehsInRange[otherVehsInRange['Vehicle id'] == neighbor_vid]['gridLoc'])
    # #     csv_k_vid.loc[csv_k_vid['Frame index'] == time, 'gridLoc'] = otherVehsInRange['Local Y'].values[0]
    
csv_k_vid.head(20)

136.0 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 207.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.]
136.99999999999997 [  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 207.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0. 

,Vehicle id,Frame index,Local X,Local Y,Lane id,Lateral_manuever,Longitudinal_manuever,v_Length,v_Width,v_Class,...,grid_115,grid_116,grid_117,grid_118,grid_119,grid_120,grid_121,grid_122,grid_123,grid_124
0,47,136.0,-2.793493,338.237904,2,1,1,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,47,137.0,-2.857777,340.407965,2,1,2,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,47,138.0,-2.866863,342.619834,2,1,2,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,47,139.0,-2.867965,344.658445,2,1,2,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,47,140.0,-2.863118,346.573577,2,1,2,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,47,141.0,-2.850364,348.375030,2,1,2,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,47,142.0,-2.833643,350.094165,2,1,2,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,47,143.0,-2.805104,351.719222,2,1,2,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,47,144.0,-2.780422,353.222747,2,1,2,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,47,145.0,-2.749790,354.602786,2,1,2,7.0,3.5,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
np.array(csv_k_vid.iloc[0])


array([ 47.        , 136.        ,  -2.79349341, 338.23790384,
         2.        ,   1.        ,   1.        ,   7.        ,
         3.5       ,   3.        ,  10.73036111,  -0.63958333,
         2.9452    ,  -2.79349341, 338.23790384,  -6.54349341,
       313.23790384,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.  

In [14]:
csv_k_vid.columns

Index(['Vehicle id', 'Frame index', 'Local X', 'Local Y', 'Lane id',
       'Lateral_manuever', 'Longitudinal_manuever', 'v_Length', 'v_Width',
       'v_Class', 'Velocity', 'Acceleration', 'azimuth', 'centVehX',
       'centVehY', 'gridMinX', 'gridMinY'],
      dtype='object')

In [ ]:
# Load data
dataset_to_use = 1
print('Loading data...')

csv = []
traj = []
for k in range(dataset_to_use):
    dataset_name = f'/home/test_usr/git_repos/project_LC/src_data2vec/dataset/preprocess/monday_d8_1000_1030_traj.csv'
    meta_name = f'/home/test_usr/git_repos/project_LC/src_data2vec/dataset/preprocess/monday_d8_1000_1030_meta.csv'
    csv_k = pd.read_csv(dataset_name)
    meta_k = pd.read_csv(meta_name)
    csv_k = preprocess_pneuma(csv_k, meta_k)
    traj_k = csv_k.iloc[:, :].values.astype('float32')
    # Add dataset id at the 1st column
    traj_k = np.concatenate([k * np.ones((traj_k.shape[0], 1), dtype='float32'), traj_k], axis=1)
    # Finally 1:dataset id, 2:Vehicle id, 3:Frame index,
    #         4:Local X, 5:Local Y, 6:Lane id.
    #         7:v_length, 8:v_Width, 9:v_Class
    #         10:Velocity (feet/s), 11:Acceleration (feet/s2).
    #         12:azimuth (rad)
    traj_k = np.concatenate([traj_k[:, :6], np.zeros((traj_k.shape[0], 2), dtype='float32'),
                              traj_k[:, 6:12], np.zeros((traj_k.shape[0], grid_cells), dtype='float32')], axis=1)

    lane_num = len(np.unique(traj_k[:, 5]))

    # # Use the vehilce's center as its location
    # offset_k = 0
    # traj_k[:, 4] = traj_k[:, 4] - 0.5 * traj_k[:, 9]
    # offset_k = np.min(traj_k[:, 4])
    # if offset_k < 0:
    #     # To make coordinate Y > 0
    #     traj_k[:, 4] = traj_k[:, 4] - offset_k

    csv.append(csv_k)
    traj.append(traj_k)

#### 2. Parse fields (listed above: maneuver label, neighbour grid)

In [ ]:
import multiprocessing as mp
import numpy as np

print('Parsing fields...')

print(f'Now process dataset {ii}')
for k in range(len(traj)):
    dsId = ii
    vehId = traj[k, 1]
    time = traj[k, 2]
    vehtraj = traj[traj[:, 1] == vehId]
    ind = np.argwhere(vehtraj[:, 2] == time).flatten()[0]
    lane = traj[k, 5]

    # Lateral maneuver in Column 7:
    ub = min(len(vehtraj), ind + 40)
    lb = max(1, ind - 40)
    if vehtraj[ub, 5] > vehtraj[ind, 5] or vehtraj[ind, 5] > vehtraj[lb, 5]:
        traj[k, 6] = 3  # Turn Right==>3.
    elif vehtraj[ub, 5] < vehtraj[ind, 5] or vehtraj[ind, 5] < vehtraj[lb, 5]:
        traj[k, 6] = 2  # Turn Left==>2.
    else:
        traj[k, 6] = 1  # Keep lane==>1.

    # Longitudinal maneuver in Column 8:
    ub = min(len(vehtraj), ind + 50)
    lb = max(1, ind - 30)
    if ub == ind or lb == ind:
        traj[k, 7] = 1  # Normal==>1
    else:
        vHist = (vehtraj[ind, 4] - vehtraj[lb, 4]) / (ind - lb)
        vFut = (vehtraj[ub, 4] - vehtraj[ind, 4]) / (ub - ind)
        if vFut / vHist < 0.8:
            traj[k, 7] = 2  # Brake==> 2
        else:
            traj[k, 7] = 1  # Normal==>1


